In [1]:
# Block 0: Imports
import os, sys
import glob
import time

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "Scripts")))

from clip_ALPOD_to_SR_extent import clip_vector_with_geometry, extract_geospatial_info_from_xml
from mask_clouds_and_classify_ice import create_mask_rasters, classify_ice_cover
from calculate_ice_cover_statistics_per_lake import calculate_lake_statistics, add_observation
from delete_incompletes_duplicates import analyze_and_delete_files

In [2]:
# Block 1: Config for lake classification

config = {
    #UDM mask bands
    'mask_bands': [3, 4, 6],
    #SR image bands to keep in the final TIF files
    'keep_bands': [3],
    'thresholds': {
        'Ice': (950, 3800),
        'Snow': (3800, float('inf')),
        'Water': (float('-inf'), 950)
    },
    'min_clear_percent': 30
}
    

study_sites_to_process = {
    'FT': [r"D:\planetscope_lake_ice\Data (CDF Testing)\Input\FT\Breakup_2019",]
}

# base output directories for individual images
rasters_output_base_dir     = r"D:\planetscope_lake_ice\Data (CDF Testing)\Output\Rasters"
shapefiles_output_base_dir  = r"D:\planetscope_lake_ice\Data (CDF Testing)\Output\Shapefiles"

# netCDF where all time series will be saved to
netcdf_output_path          = r"D:\planetscope_lake_ice\Data (CDF Testing)\Output\449.nc"

# ALPOD shapefile (wherever you downloaded it)
alpod_vector_shapefile      = r"D:\planetscope_lake_ice\Data (CDF Testing)\Input\ALPOD\ALPODlakes.shp"

In [3]:
# !git clone https://github.com/nj142/planetscope_lake_ice

In [ ]:
# Block 3: Per-Image Processing Function

def process_planetscope_image(sr_image_path, study_site):
    """
    For the given PlanetScope SR TIFF:
      1) find its accompanying UDM & XML
      2) clip the master vector by its footprint
      3) create Masked & Lake-ID rasters
      4) threshold/classify ice & snow
      5) append lake stats to NetCDF
    """
    
    # ────────────────────────────────────────────────────────────────────────────────
    # 0: Create output folders and find correlated XML & UDM for this SR image
    # ────────────────────────────────────────────────────────────────────────────────
    # identify the current season folder (e.g. “Breakup_2019”)
    season_folder = os.path.basename(os.path.dirname(sr_image_path))

    # build output dirs for this site/season

    # raster outputs (classified ice snow water, classified ALPOD id, and classified masked by cloud/nodata)
    rasters_season_dir    = os.path.join(rasters_output_base_dir, study_site, season_folder)
    classified_dir       = os.path.join(rasters_season_dir, 'Classified Rasters')
    lake_id_dir          = os.path.join(rasters_season_dir, 'Lake ID Rasters')
    masked_dir           = os.path.join(rasters_season_dir, 'Masked Rasters')
    for d in (classified_dir, lake_id_dir, masked_dir):
        os.makedirs(d, exist_ok=True)

    # shapefile outputs (clipped ALPOD lakes by image)
    shapefiles_season_dir = os.path.join(shapefiles_output_base_dir, study_site, season_folder)
    os.makedirs(shapefiles_season_dir, exist_ok=True)

    # derive the “core” image name = everything before "_ortho"
    sr_filename       = os.path.basename(sr_image_path)
    sr_name_no_ext    = os.path.splitext(sr_filename)[0]
    image_core_name   = sr_name_no_ext.split('_ortho', 1)[0]
    print(f"\n\nAnalyzing {image_core_name} in {study_site}\n# ──────────────────────────────────────────────────────────────────────────────── #")
    print("Pre-processing images for classification.")
    # locate UDM and XML metadata
    img_dir = os.path.dirname(sr_image_path)
    udm_path = os.path.join(img_dir, f"{image_core_name}_ortho_udm2.tif")
    xml_path = os.path.join(img_dir, f"{image_core_name}_ortho_analytic_4b_xml.xml")
    if not os.path.isfile(udm_path):
        raise FileNotFoundError(f"\n######################\nERROR: UDM file not found: {udm_path}\n######################\n")
    if not os.path.isfile(xml_path):
        raise FileNotFoundError(f"\n######################\nERROR: XML metadata not found: {xml_path}\n######################\n")

    # deletes any files which do not have corresponding UDM and XML files, and logs the deletion in given folder
    for study_site, paths_list in study_sites_to_process.items():
        for path in paths_list:
            analyze_and_delete_files(path)

    print("All pre-processing complete. Beginning classification.\n")

    # ────────────────────────────────────────────────────────────────────────────────
    # 1: Clip ALPOD to the image footprint (sourced from the XML file) so only lakes
    #  which fall entirely within this SR image's extent are considered
    # ────────────────────────────────────────────────────────────────────────────────
    # Initialize the subfolder to contain the shapefiles for this image (since there are multiple files like shp shx etc.)
    clip_subdir     = os.path.join(shapefiles_season_dir, sr_name_no_ext)
    os.makedirs(clip_subdir, exist_ok=True)
    clipped_vector  = os.path.join(clip_subdir, "clipped.shp")

    print("Clipping geometry...")
    geo_info = extract_geospatial_info_from_xml(xml_path)

    """clip_vector_with_geometry(
        alpod_vector_shapefile,
        geo_info['geometry'],
        clipped_vector
    )"""
    print("Geometry clipped successfully.\n")

    # ────────────────────────────────────────────────────────────────────────────────
    # 2: Mask out the land and clouds, so that we just have a masked_output raster with
    #  keep_band SR values. (For breakup, keep_band is red for red thresholding.) Also
    #  save a Lake-ID raster so we don't have to vectorize twice (saves compute power.)
    # ────────────────────────────────────────────────────────────────────────────────
    masked_output = os.path.join(masked_dir,    sr_name_no_ext + ".tif")
    lakeid_output = os.path.join(lake_id_dir,   sr_name_no_ext + ".tif")

    print("Masking out clouds and classifying raster...")
    """create_mask_rasters(
        sr_image_path,
        udm_path,
        clipped_vector,
        config['mask_bands'],
        config['keep_bands'],
        masked_output,
        lakeid_output
    )"""
    print(f"     Clouds masked successfully.")
    # ────────────────────────────────────────────────────────────────────────────────
    # 3: Classify the masked raster (we assume masked_raster contains only valid ice, 
    #  snow, and water pixels.  Misclassified clouds will be handled in time series.)
    # ────────────────────────────────────────────────────────────────────────────────
    classified_output = os.path.join(classified_dir, sr_name_no_ext + ".tif")
    """classify_ice_cover(
        masked_output,
        config['thresholds'],
        classified_output
    )"""
    print(f"Raster masked & classified successfully.\n")

    # ────────────────────────────────────────────────────────────────────────────────
    # 4: Calculate statistics for each lake, and then add the statistics to the NetCDF
    # ────────────────────────────────────────────────────────────────────────────────
    print(f"Calculating lake statistics...")
    calculate_lake_statistics(
        lakeid_output,
        classified_output,
        image_core_name,
        netcdf_output_path,
        study_site,
        alpod_vector_shapefile,
        config
    )

    print(f"Finished processing {image_core_name}\n# ──────────────────────────────────────────────────────────────────────────────── #\n")


In [ ]:
# Block 4: Loop through all images to clip, clean, and classify lake ice cover (details in script above)

for study_site, paths_list in study_sites_to_process.items():
    for site_folder in paths_list:
        pattern = os.path.join(site_folder, "*_ortho*_sr.tif")
        for sr_tif_path in glob.glob(pattern):
            try:
                process_planetscope_image(sr_tif_path, study_site)
            except Exception as e:
                print(f"Error processing {sr_tif_path}: {e}")



Analyzing 20190602_194317_0f4b in FT
# ──────────────────────────────────────────────────────────────────────────────── #
Pre-processing images for classification.
    Complete sets found: 2
    Incomplete sets found and processed: 0
    Total files deleted: 0
All pre-processing complete. Beginning classification.

Clipping geometry...
     Extracted polygon with 9 vertices
Geometry clipped successfully.

Masking out clouds and classifying raster...
     Clouds masked successfully.
Raster masked & classified successfully.

Calculating lake statistics...
